In [1]:
import sys
from pathlib import Path

# Adjust if your notebook lives in a `notebooks/` folder
project_root = Path("..").resolve() if Path(".").name == "notebooks" else Path(".").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print("Project root:", project_root)

from vector_pipeline.settings import DATA_PATH, CHROMA_DIR, COMBINED_TEXT_COLUMN
from vector_pipeline.ingestion import build_or_load_vectorstore
from vector_pipeline.retrieval import (
    preview_vectorstore,
    debug_single_retrieval,
    debug_single_rag,
    interactive_retrieval_chat,
    interactive_rag_chat,
)


Project root: /home/viktoriia/Documents/ML/project-25-2-scrum-team-data


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [2]:
import pandas as pd

df = pd.read_pickle(DATA_PATH)

print("Data path:", DATA_PATH)
print("Columns:", list(df.columns))
print("Number of rows:", len(df))

df["combined_length"] = (
    df[COMBINED_TEXT_COLUMN].fillna("").astype(str).str.len()
)

df[[COMBINED_TEXT_COLUMN, "combined_length"]].head()


Data path: data/processed_product.pkl
Columns: ['main_category', 'product_title', 'rating', 'rating_count', 'features', 'description', 'price', 'store', 'sub_categories', 'details', 'product_id', 'color', 'combined_text']
Number of rows: 3734


,combined_text,combined_length
0,Product Title: Ernie Ball Mondo Slinky Nickelw...,1532
1,Product Title: BONAOK Wireless Bluetooth Karao...,1515
2,Product Title: Boya BYM1 by Shotgun Video Micr...,1506
3,Product Title: D'Addario Guitar Strings - Phos...,1792
4,Product Title: Amazon Basics Adjustable Guitar...,820


In [3]:
vectorstore = build_or_load_vectorstore()

print("Vectorstore ready ✅")
print("Persist directory:", CHROMA_DIR)
print("Collection name:", vectorstore._collection.name)


Loading existing Chroma DB from chroma_db ...
Vectorstore loaded successfully.
Vectorstore ready ✅
Persist directory: chroma_db
Collection name: products


In [4]:
preview_vectorstore(vectorstore, k=5)


Got 5 documents from vectorstore.

=== Doc 1 ===
Metadata:
  main_category: Musical Instruments
  rating_count: 840
  price: 59.99
  store: Evenreach
  description: Description: 0
  color: Not Specified
  rating: 4.2
  product_id: B0BW8L9WF2
  product_title: Product Title: Evenreach Acoustic Panels,2 Pack 24''x12'' Premium Fabric Wrapped Panel Acoustic Wall Panels,Reduce Room Echo and Reverberation,Decorative Sound Panel for Studio/Office (Grey)
  sub_categories: Categories: Musical Instruments, Studio Recording Equipment, Studio Environment, Acoustical Treatments
  details: {'Item Weight': '2.1 pounds', 'Product Dimensions': '12 x 19685.05 x 24 inches', 'Best Sellers Rank': {'Musical Instruments': 823, 'Recording Studio Acoustical Treatments': 30}, 'Date First Available': 'June 23, 2022', 'Color Name': 'Grey', 'Size': 'Art'}
  features: Features: ☆【New & Eco-friendly】This sound absorbing panel is 24x12x0.4inches,The surface is covered with environmentally friendly fabric, polyester in

In [5]:
MANUAL_QUERY = "wireless headphones with noise cancelling"
USE_RERANKER = True

debug_single_retrieval(
    vectorstore,
    query=MANUAL_QUERY,
    k=4,
    use_reranker=USE_RERANKER,
)



Retrieved 4 documents for query: 'wireless headphones with noise cancelling'
Reranker enabled: True

──────────────────────────────────────────────
🛍️  Product Title: Alvoxcon Wireless Headset Lavalier Microphone System Wireless Lapel Mic Best for iPhone, DSLR Camera, PA Speaker, YouTube, Podcast, Video Recording, Conference, Vlogging, Church, Interview, Teaching
   ID: B07TV7J574
   Brand: Unknown brand
   Price: 49.99
   Description snippet:
   Features: SUPERIOR SOUND & PERFORMANCE – Dual uhf wireless lapel microphone delivers crystal clear sound & metal belt pack transmitter with mini receiver offer noise cancelling transmission range up to 40ft without feedback or distortion BEST CHOICE 
──────────────────────────────────────────────
──────────────────────────────────────────────
🛍️  Product Title: Alvoxcon Wireless Headset Lavalier Microphone System Wireless Lapel Mic Best for iPhone, DSLR Camera, PA Speaker, YouTube, Podcast, Video Recording, Conference, Vlogging, Church, Inter

In [6]:
RAG_QUERY = "What wireless headphones have the best battery life?"
USE_RERANKER = True

debug_single_rag(
    vectorstore,
    query=RAG_QUERY,
    k=4,
    use_reranker=USE_RERANKER,
)


RAG answer for query: 'What wireless headphones have the best battery life?'  (reranker=True)

Use ONLY the following product information to answer the question.
If the answer is not in the context, say you don't know.

CONTEXT:
Bluetooth for Wireless Music Streaming: Built-in Bluetooth for Wireless Music Streaming: Simple & Hassle-Free Pairing: Simple & Hassle-Free Pairing: Works with All of Today’s Latest Devices: Works with All of Today’s Latest Devices: Smartphones, Tablets, Laptops, Computers, etc.: Smartphones, Tablets, Laptops, Computers, etc.: Bluetooth Version: 4.0: Bluetooth Version: 4.0: Bluetooth Network Name: ‘PYLEUSA’: Bluetooth Network Name: ‘PYLEUSA’: Wireless Range: Up to 32’ ft.: Wireless Range: Up to 32’ ft.: What’s in the Box:: PA Speaker System: PA Speaker System: Remote Control: Remote Control: Power Adapter: Power Adapter: Technical Specs:: Power Output: 400 Watt MAX: Power Output: 400 Watt MAX: Sound System: 8’’ -inch Subwoofer: Sound System: 8’’ -inch Subwoofer

In [7]:
USE_RERANKER = True
interactive_retrieval_chat(vectorstore, use_reranker=USE_RERANKER)


Semantic Product Search — type 'exit' to stop.

Reranker enabled: True


Bot: I found 3 related products:

──────────────────────────────────────────────
🛍️  Product Title: Davison Guitars Full Size Electric Guitar with 10-Watt Amp, Black - Right Handed Beginner Kit with Gig Bag and Accessories
   ID: B09VVS2RYK
   Brand: Unknown brand
   Price: 119.99
   Description snippet:
   Description: If you're searching for the perfect 6-string electric guitar with a full range of accessories for a student, beginner, intermediate, or advanced guitar player, the Davison full-size electric guitar is an outstanding choice. Sold as a com
──────────────────────────────────────────────
──────────────────────────────────────────────
🛍️  Product Title: LEKATO Wireless Guitar Transmitter Receiver UHF Rechargeable Wireless Guitar System with 70 Channels Wireless Audio System (Pink)
   ID: B08MF7Q9RW
   Brand: Unknown brand
   Price: 39.99
   Description snippet:
   for most electric guitars, bass, electr

In [8]:
USE_RERANKER = True
interactive_rag_chat(vectorstore, use_reranker=USE_RERANKER)


RAG Chat — ask about any product! (type 'exit' to stop)

Reranker enabled: True


Bot: Use ONLY the following product information to answer the question.
If the answer is not in the context, say you don't know.

CONTEXT:
Description: If you're searching for the perfect 6-string electric guitar with a full range of accessories for a student, beginner, intermediate, or advanced guitar player, the Davison full-size electric guitar is an outstanding choice. Sold as a complete kit with all of the accessories, this guitar package offers excellent quality at a very attractive price. This guitar from Davison is a 39" solid body electric guitar. It features a flawless glossy finish, a maple neck with a finely finished fretboard, nickel frets, an adjustable chrome bridge, and six steel strings. It has a humbucker pickup for that rock sound, as well as two controls - one for tone, one for volume. Because it comes as a fully-equipped package, this guitar includes a 10 watt electric guitar amp with

In [ ]:
from vector_pipeline.retrieval import rag_answer

query = "what properties should I look at while choosing a good gaming computer"
answer = rag_answer(query, vs=vectorstore, k=4, use_reranker=True)
print(answer)